# Hearing Aid Audio Quality Index (HAAQI)

In summary, HAAQI is an intrusive metric that requires the processed and reference signals. In HAAQI, the reference signal is amplified by using NAL-R prescripted gain formula {cite}`byrne1986national`. HAAQI performs a seies of correlation between the reference and processed signals, therefore, it is expected that the processed signal has the same amplification. If the amplifications are not the same, the HAAQI score will be affected. Let's see this in an example.

The HAAQI function is located in `clarity/evaluator/haaqi/haaqi.py`.

The parameters are:
* processed_signal: the processed signal
* reference_signal: the reference signal
* processed_sample_rate: the sampling rate of the processed signal
* reference_sample_rate: the sampling rate of the reference signal
* audiogram: an audiogram object containing the centre frequencies and hearing loss levels in dB.
* ecualisation: a parameter that indicates if the reference signal already has the amplification applied or not.
* level1: level of the reference signal in dB SPL that corresponds to a signal RMS = 1.

In the next example, we will create a random signal and compute 2 scores:
1. The processed signal = reference signal with both having NAL-R applied
2. The processed signal = reference signal with with only the reference having NAL-R applied 

In [10]:
import numpy as np
from clarity.enhancer.nalr import NALR
from clarity.utils.audiogram import Audiogram
from clarity.evaluator.haaqi import compute_haaqi

audiogram = Audiogram(
    levels=[20, 20, 30, 30, 40, 40, 50, 60],
    frequencies=[250, 500, 1000, 2000, 3000, 4000, 6000, 8000]
)

sample_rate = 16000

enhancer = NALR(
  nfir=220,
  sample_rate=sample_rate,
)
nalr_fir, _ = enhancer.build(audiogram)

signal = np.random.randn(sample_rate * 10)

**1. Reference and processed with amplification**

In [15]:
# Apply the NAL-R amplification to each signal

processed = enhancer.apply(nalr_fir, signal)
reference = enhancer.apply(nalr_fir, signal)

# Compute HAAQI
compute_haaqi(
    processed_signal=processed,
    reference_signal=reference,
    processed_sample_rate=sample_rate,
    reference_sample_rate=sample_rate,
    audiogram=audiogram,
    equalisation=2, # reference already have amplification
    level1=65
)

0.9989907775620487

**2. Processed signal doesn't have amplification**

In [16]:
processed = signal.copy()
reference = enhancer.apply(nalr_fir, signal)

# Compute HAAQI
compute_haaqi(
    processed_signal=processed,
    reference_signal=reference,
    processed_sample_rate=sample_rate,
    reference_sample_rate=sample_rate,
    audiogram=audiogram,
    equalisation=2, # reference already have amplification
    level1=65
)

0.8512483872039563

From the example above, one can see that the best score is not exactly one but very close. Also, is the signals varies in the amplification, the score can reduce in 0.15 HAAQI in this ideal scenario.  

## Can I use other amplifications?

The short answer is Yes.

There is no reason why your HAAQI score would be negativelly affected if you use a different amplification. Just mind that you need to apply such amplification to the reference in advance (like we did) and both signals must have the same aplification.

In PyClarity, we included a multiband dynamic band compressor (MBC) that can be used for the amplification instead of NAL-R. To learn more about our MBC, please read the [MBC documentation](../signal_processing/mbc).